In [ ]:
# Installations of libraries
!pip install PyPDF2
!pip install nltk
!pip install pdfminer
!pip install tensorflow

In [ ]:
# nltk download
nltk.download('punkt')

In [102]:
# Imports
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

import io
import nltk

ModuleNotFoundError: No module named 'tensorflow'

In [83]:
# Initiate pdf file
pdf_file = "../res/who_int_0.pdf"
# Initiate dump file
temp_text = "../dump/temp.txt"

In [84]:
# https://stackoverflow.com/questions/64467930/python-pypdf2-extracttext-not-working
inFile = open(pdf_file, 'rb')
resMgr = PDFResourceManager()
retData = io.StringIO()
TxtConverter = TextConverter(resMgr, retData,laparams = LAParams())
interpreter = PDFPageInterpreter(resMgr, TxtConverter)

# Process each pages
for page in PDFPage.get_pages(inFile): 
    interpreter.process_page(page)

# Write to text temporary file
txt = retData.getvalue()
with open(temp_text, 'w') as f: 
     f.write(txt)

In [85]:
raw = open(outTXTFile).read()

In [86]:
# Length of raw text file
len(raw)

7876

In [87]:
# Type of raw text file
type(raw)

str

In [100]:
# Initialize Tokenizer
tokens = nltk.word_tokenize(raw)

In [101]:
tokenizer = Tokenizer(raw)

NameError: name 'Tokenizer' is not defined

In [90]:
# Length of tokenized text file
len(tokens)

1353

In [96]:
# Type of tokenized text file
type(tokens)

list

In [97]:
# Sample print of list
print(tokens[0:69])

['3/15/22', ',', '10:32', 'AM', 'Is', 'the', 'pandemic', 'ending', 'soon', '?', 'Western', 'Pacific', 'Philippines', '©', 'Opinion', ':', 'Is', 'the', 'pandemic', 'ending', 'soon', '?', '11', 'March', '2022', 'Statement', '|', 'By', 'Dr', 'Rajendra', 'Yadav', 'and', 'Dr', 'Sangjun', 'Moon', ',', 'WHO', 'Philippines', 'Manila', ',', 'Philippines', '(', '11', 'March', '2022', ')', '–', 'When', 'will', 'this', 'end', '?', 'This', 'question', 'has', 'been', 'on', 'everyone', '’', 's', 'mind', 'for', 'many', 'months', '.', 'We', 'are', 'all', 'tired']
